# Stock Pattern Analysis - Main Notebook

Google Colab上でGPU加速された株価パターンマッチング分析を実行します。

## 📋 実行手順

1. **Secrets設定**: 左サイドバーの🔑から以下を設定
   - `GITHUB_TOKEN`: GitHub Personal Access Token
   - `STOCK_JSON_FOLDER`: JSONデータのパス
   - `STOCK_RESULTS_FOLDER`: 結果保存先

2. **セルを順番に実行**: Runtime > Run all または各セルを実行

3. **結果確認**: グラフがColab内に表示され、デバッグモードONの場合はGitHubにもアップロード

## Step 1: 環境セットアップ

In [ ]:
# Google Drive マウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Colab Secrets 読み込み
from google.colab import userdata

GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
STOCK_JSON_FOLDER = userdata.get('STOCK_JSON_FOLDER')
STOCK_RESULTS_FOLDER = userdata.get('STOCK_RESULTS_FOLDER')
REPORTS_FOLDER = userdata.get('REPORTS_FOLDER', 'reports')

print(f"✅ Secrets loaded successfully")
print(f"   Data folder: {STOCK_JSON_FOLDER}")
print(f"   Results folder: {STOCK_RESULTS_FOLDER}")

In [ ]:
# リポジトリクローン
!git clone https://github.com/kabuist001/stock-pattern-matcher.git
%cd stock-pattern-matcher

In [ ]:
# 依存関係インストール
!pip install -q -r requirements.txt
print("✅ Dependencies installed")

## Step 2: データ読み込み

In [ ]:
from src.data_loader import StockDataLoader
from utils.config import get_config

config = get_config()

print("📂 Loading stock data...")
loader = StockDataLoader(STOCK_JSON_FOLDER)
symbols_data = loader.load_all_data()

summary = loader.get_summary()
display(summary.head(20))

print(f"\n✅ Loaded {len(symbols_data)} symbols")

## Step 3: GPU対応パターンマッチング分析

In [ ]:
from src.pattern_matcher_gpu import PatternMatcher

matcher = PatternMatcher(
    window_size=config.window_size,
    lookahead=config.lookahead,
    top_n=config.top_n,
    min_similarity=config.min_similarity,
    use_gpu=config.gpu_enabled
)

print("\n🔍 Starting pattern matching analysis...")
results = matcher.analyze_all_symbols(symbols_data)

import os
os.makedirs(STOCK_RESULTS_FOLDER, exist_ok=True)
results_path = f"{STOCK_RESULTS_FOLDER}/pattern_results.csv"
results.to_csv(results_path, index=False)
print(f"\n💾 Results saved to: {results_path}")

display(results.head(20))

## Step 4: 可視化

In [ ]:
from src.visualizer import Visualizer

viz = Visualizer(config)
plots = viz.create_all_plots(results, symbols_data)

stats = viz.create_summary_stats(results)
print("\n📊 Analysis Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

In [ ]:
# グラフをColab内で表示
for plot_name, fig in plots.items():
    print(f"\n📈 {plot_name}")
    fig.show()

## Step 5: HTMLレポート生成とGitHubアップロード

In [ ]:
from src.report_generator import ReportGenerator
from utils.github_manager import GitHubManager

report_gen = ReportGenerator(config)
html_report = report_gen.generate(results, plots, stats)

print("✅ HTML report generated")

if config.debug_mode:
    print("\n📤 Debug mode ON - Uploading report to GitHub...")
    gh = GitHubManager(GITHUB_TOKEN, config)
    if gh.check_repository_access():
        report_url = gh.upload_report(html_report)
        if report_url:
            print(f"\n🎉 Report uploaded successfully!")
            print(f"   View report: {report_url}")
            print(f"\n💡 Share this URL with Claude for analysis and improvements")
        else:
            print("\n⚠️  Failed to upload report")
    else:
        print("\n⚠️  Cannot access GitHub repository")
else:
    print("\n📊 Debug mode OFF - Report displayed in Colab only")
    from IPython.display import HTML
    display(HTML(html_report))